In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')

In [2]:
train['Demanda_uni_equil_log'] = np.log1p(train['Demanda_uni_equil'])

In [3]:
test['pred'] = np.nan

In [4]:
mean_prod = train.groupby(['Producto_ID'])['Demanda_uni_equil_log'].mean().reset_index()
mean_prod_agenc_client = train.groupby(['Producto_ID', 'Agencia_ID', 'Cliente_ID'])['Demanda_uni_equil_log'].mean().reset_index()
mean_prod_ruta = train.groupby(['Producto_ID', 'Ruta_SAK'])['Demanda_uni_equil_log'].mean().reset_index()

In [5]:
test = pd.merge(test, mean_prod, how='left', on=['Producto_ID'])
test.rename(columns={'Demanda_uni_equil_log': 'prod'}, inplace=True)
test = pd.merge(test, mean_prod_ruta, how='left', on=['Producto_ID', 'Ruta_SAK'])
test.rename(columns={'Demanda_uni_equil_log': 'prod_ruta'}, inplace=True)
test = pd.merge(test, mean_prod_agenc_client, how='left', on=['Producto_ID', 'Agencia_ID', 'Cliente_ID'])
test.rename(columns={'Demanda_uni_equil_log': 'prod_age_cli'}, inplace=True)

In [8]:
np.expm1(test[['prod', 'prod_ruta', 'prod_age_cli']].mean(axis=1)).to_csv('sub1.csv')

In [34]:
submission = []
for row in test.iterrows():
    if np.isnan(row[1]['prod_age_cli']):
        if np.isnan(row[1]['prod_ruta']):
            if np.isnan(row[1]['prod']):
                submission.append(1)
            else:
                submission.append(row[1]['prod']) 
        else:
            submission.append(row[1]['prod_ruta'])
    else:
        submission.append(row[1]['prod_age_cli'])

In [28]:
(0.91*np.expm1(pd.Series(submission))).to_csv('sub2.csv')

In [35]:
mean_pred = np.expm1(test[['prod', 'prod_ruta', 'prod_age_cli']].mean(axis=1))
order_pred = 0.91*np.expm1(pd.Series(submission))

In [36]:
order_pred.fillna(1).to_csv('sub2.csv')

In [33]:
def RMSLE_score(pred, true, inds):
    score = 0
    for ind in inds:
        score += (np.log1p(pred[ind]) - np.log1p(true[ind]))**2
    score = np.sqrt(score/len(inds))
    return score

In [56]:
inds = np.random.choice(range(len(train)), int(0.01*len(train)))
RMSLE_score(0.9*ptrain, train['Demanda_uni_equil'].values, inds)

0.48417830792395533

In [32]:
mean_prod_agenc_client = train.groupby(['Producto_ID', 'Agencia_ID', 'Cliente_ID'])['Demanda_uni_equil_log'].mean()

In [46]:
ruta = train.groupby(['Producto_ID', 'Ruta_SAK'])['Demanda_uni_equil_log'].mean()

In [25]:
pd.Series(submission)

0          1.609438
1          1.006477
2          1.098612
3          1.059591
4          1.310750
5          1.460676
6          0.828302
7          1.892663
8          1.907352
9          1.953121
10         2.246999
11         1.098612
12         1.629048
13         2.293817
14         1.177221
15         1.716351
16         0.982927
17         1.133732
18         0.895880
19         1.609438
20         1.940755
21         2.499225
22         2.469455
23         0.965939
24         0.693147
25         2.113276
26         1.267526
27         1.806634
28         0.693147
29         0.693147
             ...   
6999221    1.373032
6999222    1.310437
6999223    1.703766
6999224    0.895880
6999225    2.596058
6999226    1.464816
6999227    1.984002
6999228    1.229845
6999229    1.415922
6999230    4.286372
6999231    1.098612
6999232    1.364782
6999233    0.972206
6999234    0.822406
6999235    1.916859
6999236    1.010298
6999237    1.395786
6999238    1.945910
6999239    1.975122


In [6]:
train['Producto_ID'].unique()

array([ 1212,  1216,  1238, ..., 35441, 35442, 42192], dtype=int64)